In [1]:
import pandas as pd
import itertools

In [2]:
data=pd.read_csv('test_stripped.csv')

In [4]:
groups=data[['skill_category', 'no_stopwords_stemmed']]

In [5]:
d_group=groups.groupby('skill_category')['no_stopwords_stemmed'].apply(' '.join).reset_index()

In [6]:
frequency_dict=[dict(pd.Series(i.split()).value_counts()) for i in d_group.no_stopwords_stemmed]

In [7]:
top_profile_words={}
for i,k in zip(d_group.skill_category,frequency_dict):
    top_profile_words[i]=[p for p,s in k.items() if s>500]

In [8]:
%%time
skill_ranks=[]
ids=[]
skills=[]
for j,i in enumerate(data.no_stopwords_stemmed):
    for k in top_profile_words:
        ids.append(j)
        skills.append(k)
        skill_ranks.append(len(list(set([p for p in i.split() if p in top_profile_words[k]])))/len(k))

CPU times: user 2min 12s, sys: 21 ms, total: 2min 12s
Wall time: 2min 12s


In [9]:
df=pd.concat([pd.Series(ids, name='id'),pd.Series(skills, name='skill_category'), 
              pd.Series(skill_ranks, name='skill_category_alignment')], axis=1)

df['skill_category']=[i.replace('.xls','') for i in df['skill_category']]

In [11]:
%%time
df_sorted=[]
for i in range(0,len(df.id.unique())):
    df_sorted.append(df[df.id==i].sort_values(['skill_category_alignment'], ascending=False))

CPU times: user 19.4 s, sys: 44 ms, total: 19.4 s
Wall time: 19.4 s


In [12]:
%%time
sorted_ids=[]
sorted_skill_categories=[]
sorted_rank=[]
for i in df_sorted:
    i=i.reset_index()
    sorted_ids.append(['ID_'+str(p) for p in i.id])
    sorted_skill_categories.append(i.skill_category)
    sorted_rank.append(list(range(1,len(i)+1)))

CPU times: user 8.24 s, sys: 40 ms, total: 8.28 s
Wall time: 8.28 s


In [13]:
%%time
df_full=pd.concat([pd.Series(list(itertools.chain.from_iterable([list(i) for i in sorted_ids])), name='ids'),
                   pd.Series(list(itertools.chain.from_iterable([list(i) for i in sorted_skill_categories])), 
                             name='skill_category'),
                  pd.Series(list(itertools.chain.from_iterable([list(i) for i in sorted_rank])), 
                            name='rank')], axis=1)

CPU times: user 378 ms, sys: 16 ms, total: 394 ms
Wall time: 391 ms


In [14]:
#df_full.to_csv('df_full.csv')

In [16]:
df_full10000=df_full.iloc[:10000,:]
#df_full10000.to_csv('df_full_10000.csv')